In [7]:
#pip install beautifulsoup4
#!pip install lxml
#!pip install html5lib
#!pip install requests

<h2>1) Scraping Wikipedia and Preprocessing Data</h2>

In [2]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.content)

table = soup.find('table')
rows = table.find_all('tr')
data_df = []
#df = pd.DataFrame(data, columns = ['Postal code', 'Borough','Neighborhood']) 

for row in rows:
    data = row.find_all("td")
    if len(data) > 0:
        if(data[1].get_text() != 'Not assigned\n'):
            replace_str =  data[2].get_text().strip('\n').replace("/", ",")
            data_df.append([data[0].get_text().strip('\n'),data[1].get_text().strip('\n'),data[2].get_text().strip('\n')])

df = pd.DataFrame(data_df, columns = ['Postal Code', 'Borough','Neighborhood']) 

In [3]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [4]:
df.shape

(103, 3)

<h2>2) Merge GeoSpatial data with Postal Code</h2>

In [5]:
geo_data = pd.read_csv("http://cocl.us/Geospatial_data") 
geo_data

geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
df_merge = pd.merge(df, geo_data, on='Postal Code', how='inner')
df_merge.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


<h2>3) Analysis quantitive of venues each Neighborhood</h2>

In [10]:
CLIENT_ID = '3OWMTFK5RR4LBS1IQHP3M3SAPDR5CMWTMNWEWIGXSI5QODYU'
CLIENT_SECRET = 'DIH05XTGILACFG2VKGAZS4AZOWN32GOPFERR11Z4XGG1FSKX'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
VERSION = '20180605' # Foursquare API version
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
toronto_venues = getNearbyVenues(names=df_merge['Neighborhood'],
                                   latitudes=df_merge['Latitude'],
                                   longitudes=df_merge['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [13]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [22]:
ds_toronto = toronto_venues.groupby('Neighborhood').Venue.nunique()
ds_toronto.to_frame()
df_forvisualize = pd.merge(df_merge, ds_toronto, on='Neighborhood', how='left')
df_forvisualize.fillna(0, inplace=True)
df_forvisualize.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,5.0
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,46.0
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,16.0
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,26.0


In [25]:
bins = [-1, 25, 50, 75, 101]
group = ['1', '2', '3', '4']
df_forvisualize['groupcluster'] = pd.cut(df_forvisualize['Venue'], bins, labels=group)
df_forvisualize.head(8)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Venue,groupcluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,1
1,M4A,North York,Victoria Village,43.725882,-79.315572,5.0,1
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,46.0,2
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,16.0,1
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,26.0,2
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,0.0,1
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,2.0,1
7,M3B,North York,Don Mills,43.745906,-79.352188,25.0,1


In [29]:
#!pip install geopy
from geopy.geocoders import Nominatim 
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [34]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_forvisualize['Latitude'], df_forvisualize['Longitude'], df_forvisualize['Neighborhood'], df_forvisualize['groupcluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters